In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pprint
from uuid import UUID
import time
import gspread_dataframe as gsdf

In [2]:
class gspread_Operator:
    __scope: list[str]
    __key_file: str
    
    def __init__(self):
        self.__scope.append('https://www.googleapis.com/auth/drive')
        self.__scope.append('https://www.googleapis.com/auth/drive.file')
        self.__key_file = 'main-webserver-clientkey.json'
        
    def authorize(self):
        creds = ServiceAccountCredentials.from_json_keyfile_name(f'../keys/{self.__key_file}',self.__scope)
        return gspread.authorize(creds)

    def write_headlines(gs_client, validated_titles):
        # Writes a set of validated headlines to a Google sheet
        #gs_client = activate()
        file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
        worksheet = file.worksheet('Links')  # Headline are written to this sheet
        lines = 0
        for idx, row in validated_titles.iterrows():
            if(isinstance(row['uuid'], UUID)):
                row['uuid'] = str(row['uuid'])
            worksheet.append_row(row.tolist())
            lines += 1 
        print(f'Wrote {lines} lines to Links sheet in the workbook')
        worksheet.client.session.close()
        return 0

    def write_headlines_in_chunks(self, gs_client, validated_titles, delay=1, chunk_size=20):
        ## Write data to GSheets slowly for large datasets because of throttling
        for i in range(0, len(validated_titles), chunk_size):
            self.write_headlines(gs_client, validated_titles.iloc[i:i+chunk_size])
            time.sleep(delay)
        print(f'Wrote all {len(validated_titles)} lines of titles to file.')
        return 0

    def write_recommendations(gs_client, recos):
        # Writes a set of article Recommendations to the google sheet
        #gs_client = activate()
        file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
        worksheet = file.worksheet('Recos')  # Recos are written to this sheet
        lines = 0
        for idx, row in recos.iterrows():
            if(isinstance(row['uuid'], UUID)):
                row['uuid'] = str(row['uuid'])
            worksheet.append_row(row.tolist())
            lines += 1
        print(f'Wrote {lines} lines to Recos  sheet in the workbook')
        worksheet.client.session.close()
        return 0

    def write_recommendations_in_chunks(self, gs_client, recos, delay=1, chunk_size=20):
        ## Write data to GSheets slowly for large datasets because of throttling
        for i in range(0, len(recos), chunk_size):
            self.write_headlines(gs_client, recos.iloc[i:i+chunk_size])
            time.sleep(delay)
        print(f'Wrote all {len(recos)} lines of titles to file.')
        return 0

    def write_et_link(gs_client, articles):
        # Writes a set of article Recommendations to the google sheet
        #gs_client = activate()
        file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
        worksheet = file.worksheet('ET_Links')  # ET Headline are written to this sheet
        lines = 0 
        for idx, row in articles.iterrows():    #This mechanism of row_wise appending will fail for more than 60 rows as Google rate limits the number of Write request that can be made in a minute. Better we write as a chunk 
            if(isinstance(row['uuid'], UUID)):
                row['uuid'] = str(row['uuid'])
            worksheet.append_row(row.tolist())
            lines += 1
        print(f'Wrote {lines} lines to ET Links sheet in the workbook')
        worksheet.client.session.close()
        return 0

    def read_latest_recos(gs_client):
        # Reads the last written recos from the sheet
        #gs_client = activate()
        file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
        worksheet = file.worksheet('Recos')  # Recos are written to this sheet
    
        recos = gsdf.get_as_dataframe(worksheet=worksheet, header=0, nrows=None)
        last_date = recos['art_date'].max()
        recos = recos[recos['art_date'] == last_date]
        recos.drop(['uuid', 'user_review'], axis=1, inplace=True)
        
        worksheet.client.session.close()
        return recos

In [3]:
def activate():
    scope = [
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
        ]
    key_file_name = 'main-webserver-clientkey.json'
    creds = ServiceAccountCredentials.from_json_keyfile_name(f'../keys/{key_file_name}',scope)
    return gspread.authorize(creds)

In [4]:
def write_headlines(validated_titles):
    # Writes a set of validated headlines to a Google sheet
    
    gs_client = activate()
    file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
    worksheet = file.worksheet('Links')  # Headline are written to this sheet
    lines = 0

    for idx, row in validated_titles.iterrows():
        if(isinstance(row['uuid'], UUID)):
            row['uuid'] = str(row['uuid'])
        worksheet.append_row(row.tolist())
        lines += 1 
    print(f'Wrote {lines} lines to Links sheet in the workbook')
    worksheet.client.session.close()
    return 0

In [5]:
def write_recommendations(recos):
    # Writes a set of article Recommendations to the google sheet
    
    gs_client = activate()
    file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
    worksheet = file.worksheet('Recos')  # Recos are written to this sheet
    lines = 0

    for idx, row in recos.iterrows():
        if(isinstance(row['uuid'], UUID)):
            row['uuid'] = str(row['uuid'])
        worksheet.append_row(row.tolist())
        lines += 1

    print(f'Wrote {lines} lines to Recos  sheet in the workbook')
    worksheet.client.session.close()
    return 0

In [6]:
def write_et_link(articles):
    # Writes a set of article Recommendations to the google sheet
    
    gs_client = activate()
    file = gs_client.open_by_key('1X1nGWEdD2ExW1Z9i1cX5gCpZjYmAFc14GzmoKmnJYU0')  # All writes are to this specific file
    worksheet = file.worksheet('ET_Links')  # ET Headline are written to this sheet
    lines = 0 
    
    for idx, row in articles.iterrows():    #This mechanism of row_wise appending will fail for more than 60 rows as Google rate limits the number of Write request that can be made in a minute. Better we write as a chunk 
        if(isinstance(row['uuid'], UUID)):
            row['uuid'] = str(row['uuid'])
        worksheet.append_row(row.tolist())
        liones += 1

    print(f'Wrote {lines} lines to ET Links sheet in the workbook')

    worksheet.client.session.close()
    return 0

In [7]:
#one line tester

#write_headlines([]);